In [ ]:
# Training parameters

window = 672       # 1 week
stride = 4         # 1 hour
latent_dim = 10    # Autoencoder latent dimension
epochs = 150       # Number of epochs
batch_size = 8     # Batch size
M = 200            # Montecarlo

In [ ]:
class SimScore(tfk.layers.Layer):

    def call(self, inputs):
        seq, h_dim = inputs  # seq: batch x x_dim x h_dim*2

        S = tf.map_fn(fn=lambda t: tf.linalg.matmul(tf.transpose(t), t), elems=seq) # batch x h_dim x h_dim
        S = tf.map_fn(fn=lambda t: t / tf.math.sqrt((tf.cast(h_dim*2, dtype=tf.float32))), elems=S)
        A = tf.map_fn(fn=lambda t: tf.keras.activations.softmax(t), elems=S)
        C = tf.matmul(seq, A)

        return C

In [ ]:
class AddNoise(tfk.layers.Layer):
    
    def call(self, inputs):
        data, noise_factor = inputs
        noise = tf.map_fn(fn=lambda t: tf.random.normal((672,2), 0, 1)*noise_factor, elems=data)
        noise_input = data + noise
        
        return noise_input

In [ ]:
# Building the model

input_shape = X_train.shape[1:]
output_shape = X_train.shape[1:]

from keras import backend as K
from tensorflow.keras import Input

###########
# ENCODER #
###########

encoder_input = tf.keras.Input(shape=input_shape)

h_seq, forward_h, forward_c = tfkl.LSTM(64, return_sequences=True, return_state=True)(encoder_input)

Cdet = SimScore()([h_seq, window])

c_mean = tfkl.Dense(window, activation='linear', name="c_mean")(Cdet)
c_log_var = tfkl.Dense(window, activation='softplus', name="c_var")(Cdet)

# Latent representation: mean + log of std.dev.
z_mean = tfkl.Dense(latent_dim, activation='linear', name="z_mean")(forward_h)
z_log_var = tfkl.Dense(latent_dim, activation='linear', name="z_var")(forward_h)

# Reparametrization trick
def sample_z1(args):
    z_mean, z_log_var = args
    eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1]))
    return z_mean + tf.exp(alpha * z_log_var) * eps

# Reparametrization trick
def sample_z2(args):
    z_mean, z_log_var = args
    eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1], K.int_shape(z_mean)[2]))
    return z_mean + tf.exp(alpha * z_log_var) * eps
    
# Sampling a vector from the latent distribution
z = tfkl.Lambda(sample_z1, name='z')([z_mean, z_log_var])
c = tfkl.Lambda(sample_z2, name='c')([c_mean, c_log_var])

encoder = tfk.Model(encoder_input, [z_mean, z_log_var, z, c_mean, c_log_var, c], name='encoder')
print(encoder.summary())

In [ ]:
###########
# DECODER #
###########

z_inputs = Input(shape=(latent_dim, ), name='decoder_input_1')
c_inputs = Input(shape=(window, window), name='decoder_input_2')

repeated = tf.transpose(tfkl.RepeatVector(window)(z_inputs), perm = [0, 2, 1])
concat = tf.transpose( tfkl.Concatenate(axis=1)([repeated, c_inputs]), perm = [0, 2, 1] )

x = tfkl.LSTM(64, return_sequences=True)(concat)
x = tfkl.TimeDistributed(tfkl.Dense(output_shape[1]))(x)

mu = tfkl.Conv1D(X_train.shape[2],2,1, padding='same', name='mu')(x)
sigma = tfkl.Conv1D(X_train.shape[2],2,1, padding='same', name='sigma')(x)


# REPRESENTATION USED ONLY FOR GRAPHICAL PURPOSES

# Reparametrization trick
def sample_z2(args):
    z_mean, z_log_var = args
    eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1], K.int_shape(z_mean)[2]))
    return z_mean + tf.exp(alpha * z_log_var) * eps

decoder_output = tfkl.Lambda(sample_z2, output_shape=(672, 2), name='decoder_output')([mu, sigma])

# Define and summarize decoder model
decoder = tfk.Model([z_inputs, c_inputs], [mu, sigma, decoder_output], name='decoder')
decoder.summary()

In [ ]:
class VAE(tfk.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tfk.metrics.Mean(name="total_loss")
        self.likelihood_tracker = tfk.metrics.Mean(name="likelihood")
        self.kl_loss_z_tracker = tfk.metrics.Mean(name="kl_loss_z")
        self.kl_loss_c_tracker = tfk.metrics.Mean(name="kl_loss_c")
        self.reconstruction_loss_tracker = tfk.metrics.Mean(name="reconstruction_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.likelihood_tracker,
            self.kl_loss_z_tracker,
            self.kl_loss_c_tracker,
            self.reconstruction_loss_tracker
        ]
    

    def train_step(self, data):
        with tf.GradientTape() as tape:
            
            # Reparametrization trick
            def sample_z2(args):
                z_mean, z_log_var = args
                eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1], K.int_shape(z_mean)[2]))
                return z_mean + tf.exp(alpha * z_log_var) * eps
            
            encoder_mu, encoder_log_var, z, c_mean, c_log_var, c = self.encoder(data)
            decoder_mu, decoder_log_var, _ = self.decoder([z,c])
            decoder_sigma = tf.exp(decoder_log_var)
                             
            pdf_normal = tfp.distributions.MultivariateNormalDiag(decoder_mu, decoder_sigma, validate_args=True, name='Gauss')
            likelihood = -(pdf_normal.log_prob(data))
            likelihood = tf.reduce_mean(likelihood, axis=-1)
            likelihood = tf.reduce_mean(likelihood, axis=-1)
                
            decoder_output = tfkl.Lambda(sample_z2, output_shape=input_shape, name='decoder_output')([decoder_mu, decoder_log_var])
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, decoder_output), axis=1))
            
            kl_loss_z = -0.5 * (1 + encoder_log_var - tf.square(encoder_mu) - tf.exp(encoder_log_var))
            kl_loss_z = tf.reduce_mean(tf.reduce_sum(kl_loss_z, axis=1))
            
            kl_loss_c = -0.5 * (1 + c_log_var - tf.square(c_mean) - tf.exp(c_log_var))
            kl_loss_c = tf.keras.backend.sum(kl_loss_c, axis=1)
            kl_loss_c = tf.reduce_mean(kl_loss_c, axis=1)

            total_loss = 3*likelihood + kl_loss_z + 0.5*kl_loss_c + reconstruction_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.likelihood_tracker.update_state(likelihood)
        self.kl_loss_z_tracker.update_state(kl_loss_z)
        self.kl_loss_c_tracker.update_state(kl_loss_c)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "likelihood": self.likelihood_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result()
        }
    
    
    
    def test_step(self, data): # https://github.com/keras-team/keras-io/issues/38

        # Reparametrization trick
        def sample_z2(args):
            z_mean, z_log_var = args
            eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1], K.int_shape(z_mean)[2]))
            return z_mean + tf.exp(alpha * z_log_var) * eps
            
        encoder_mu, encoder_log_var, z, c_mean, c_log_var, c = self.encoder(data)
        decoder_mu, decoder_log_var, _ = self.decoder([z,c])
        decoder_sigma = tf.exp(decoder_log_var)
                             
        pdf_normal = tfp.distributions.MultivariateNormalDiag(decoder_mu, decoder_sigma, validate_args=True, name='Gauss')
        likelihood = -(pdf_normal.log_prob(data))
        likelihood = tf.reduce_mean(likelihood, axis=-1)
        likelihood = tf.reduce_mean(likelihood, axis=-1)
                
        decoder_output = tfkl.Lambda(sample_z2, output_shape=input_shape, name='decoder_output')([decoder_mu, decoder_log_var])
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, decoder_output), axis=1))
            
        kl_loss_z = -0.5 * (1 + encoder_log_var - tf.square(encoder_mu) - tf.exp(encoder_log_var))
        kl_loss_z = tf.reduce_mean(tf.reduce_sum(kl_loss_z, axis=1))
            
        kl_loss_c = -0.5 * (1 + c_log_var - tf.square(c_mean) - tf.exp(c_log_var))
        kl_loss_c = tf.keras.backend.sum(kl_loss_c, axis=1)
        kl_loss_c = tf.reduce_mean(kl_loss_c, axis=1)

        total_loss = 3*likelihood + kl_loss_z + 0.5*kl_loss_c + reconstruction_loss
            
        self.total_loss_tracker.update_state(total_loss)
        self.likelihood_tracker.update_state(likelihood)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_z_tracker.update_state(kl_loss_z)
        self.kl_loss_c_tracker.update_state(kl_loss_c)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "likelihood": self.likelihood_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result()
        }

In [ ]:
# Training
vae = VAE(encoder, decoder)
vae.compile(optimizer=tfk.optimizers.Adam())
vae.fit(x = X_train,
        validation_data = (X_val, None),
        epochs=epochs, 
        batch_size=batch_size,
        callbacks=[
            tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
       )